In [ ]:
import adcc
import psi4
from matplotlib import pyplot as plt

adcc.set_n_threads(4)  # Set to 4 threads

In [ ]:
# Run SCF in Psi4
mol = psi4.geometry("""
    units au
    symmetry c1
    O 0 0 0
    H 0 0 1.795239827225189
    H 1.693194615993441 0 -0.599043184453037
""")
psi4.core.be_quiet()
psi4.set_options({'basis': "cc-pvtz", 'e_convergence': 1e-7})
_, wfn = psi4.energy('SCF', return_wfn=True)

In [ ]:
# Run ADC(1) on 7 singlets
n_singlets = 7
adc_1 = adcc.adc1(wfn, n_singlets=n_singlets)

In [ ]:
# Build guess for ADC(2) by appending a zero doubles part to each vector
adc2matrix = adcc.AdcMatrix("adc2", adc_1.ground_state)
def append_doubles_block(adc1_vector):
    symmetries = adcc.guess_symmetries(adc2matrix,
                                       spin_block_symmetrisation="symmetric")
    return adcc.AmplitudeVector(adc1_vector["s"], adcc.Tensor(symmetries[1]))
guesses_1 = [append_doubles_block(exv) for exv in adc_1.excitation_vectors]

In [ ]:
# Run ADC(2), ADC(2x) and ADC(3)
adc_2 =  adcc.adc2(adc2matrix, n_singlets=n_singlets, guesses=guesses_1)
adc_x = adcc.adc2x(adc_2.ground_state, n_singlets=n_singlets,
                     guesses=adc_2.excitation_vectors)
adc_3 =  adcc.adc3(adc_x.ground_state, n_singlets=n_singlets,
                     guesses=adc_x.excitation_vectors)

In [ ]:
adc_1.plot_spectrum(label="ADC(1)")
adc_2.plot_spectrum(label="ADC(2)")
adc_x.plot_spectrum(label="ADC(2)-x")
adc_3.plot_spectrum(label="ADC(3)")
plt.legend()